In [ ]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests siphon

In [3]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import subprocess
import sys
from siphon.simplewebservice.ndbc import NDBC
import requests, pandas as pd
from datetime import datetime, timedelta, timezone, time
import os,json
from pathlib import Path
import plotly as pt
import pytz, folium, gspread
from datetime import datetime
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from requests.exceptions import HTTPError  # Importer HTTPError
import sys
from urllib.error import HTTPError
import dask
from sklearn.preprocessing import StandardScaler
from siphon.simplewebservice.ndbc import NDBC
import psycopg2
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, TIMESTAMP, text
from sqlalchemy.exc import ProgrammingError
from urllib.parse import quote_plus

"""
def install_and_import(*packages):
    for package in packages:
        try:
            # Tenter d'importer chaque bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' est déjà installé et importé.")
        except ImportError:
            # Si l'import échoue, installer la bibliothèque
            print(f"Le package '{package}' n'est pas installé. Installation en cours...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            # Ensuite, essayer à nouveau d'importer la bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' a été installé et importé.")
 
            
# Utilisation de la fonction pour installer et importer plusieurs packages
install_and_import("dask", "pyowm", "meteomatics","ndbc_api", "plotly", "geopy")

"""
import plotly.express as px
import re

Fonctions

In [56]:
def drop_columns_if_exist(df, columns_to_drop):
    existing_columns = []
    for col in columns_to_drop:
        if col in df.columns:
            existing_columns.append(col)
            print(f"Colonne '{col}' Supprimée")
        else: 
            print(f"Colonne '{col}' Non Trouvée")
    return df.drop(columns=existing_columns)

def add_daytime_and_month_column(df, time_column='Datetime'):
    """
    Ajoute les colonnes 'DayTime' et 'Month' basées sur l'heure et le mois de la colonne 'Datetime' ou d'une autre colonne de type datetime.
    
    Parameters:
    df (pd.DataFrame): La DataFrame sur laquelle ajouter les colonnes.
    time_column (str): Nom de la colonne ou 'index' pour utiliser l'index datetime. Par défaut, utilise l'index.
    
    Returns:
    pd.DataFrame: DataFrame avec les nouvelles colonnes 'DayTime' et 'Month'.
    """
    # Vérifier si la colonne choisie est valide
    if time_column == 'index':
        time_data = df.index
    elif time_column in df.columns and pd.api.types.is_datetime64_any_dtype(df[time_column]):
        time_data = df[time_column]
    else:
        raise ValueError("La colonne spécifiée n'est pas valide ou n'est pas de type datetime.")
    
    # Fonction pour déterminer la période de la journée
    def get_daytime(hour):
        if 6 <= hour < 12:
            return "Morning"
        elif 12 <= hour < 18:
            return "Afternoon"
        elif 18 <= hour < 21:
            return "Evening"
        else:
            return "Night"
    
    # Si on utilise l'index (DatetimeIndex)
    if isinstance(time_data, pd.DatetimeIndex):
        # Extraire l'heure et le mois de l'index datetime
        df['DayTime'] = time_data.hour.map(lambda hour: get_daytime(hour))
        df['Month'] = time_data.month
    elif isinstance(time_data, pd.Series) and pd.api.types.is_datetime64_any_dtype(time_data):
        # Si c'est une colonne datetime classique
        df['DayTime'] = time_data.dt.hour.map(lambda hour: get_daytime(hour))
        df['Month'] = time_data.dt.month
    else:
        raise ValueError("Les données temporelles ne sont pas de type datetime.")

    return df
    
    # Fonction pour déterminer la période de la journée
    def get_daytime(hour):
        if 6 <= hour < 12:
            return "Morning"
        elif 12 <= hour < 18:
            return "Afternoon"
        elif 18 <= hour < 21:
            return "Evening"
        else:
            return "Night"
    # Si on utilise l'index (DatetimeIndex)
    if isinstance(time_data, pd.DatetimeIndex):
        # Extraire l'heure et le mois de l'index datetime
        df['DayTime'] = time_data.hour.map(lambda hour: get_daytime(hour))
        df['Month'] = time_data.month
    else:
        # Si on utilise une colonne datetime classique
        df['DayTime'] = time_data.dt.hour.map(lambda hour: get_daytime(hour))
        df['Month'] = time_data.dt.month
    
    return df

def process_and_resample(df, column_name, resample_interval='h'):
    try:
        # Vérification si la colonne existe dans le DataFrame
        if column_name not in df.columns:
            print(f"Erreur : La colonne '{column_name}' n'existe pas dans le DataFrame.")
            return df

        # Renommer la colonne spécifiée en 'Datetime' et la convertir en datetime
        df.rename(columns={column_name: 'Datetime'}, inplace=True)
        df['Datetime'] = pd.to_datetime(df['Datetime'], errors='raise')

        # Valider l'intervalle de resampling
        valid_intervals = ['h', 'd', 'm', 'w', 'M', 'Y']
        if resample_interval not in valid_intervals:
            print(f"Erreur : L'intervalle de resampling '{resample_interval}' n'est pas valide. Utilisez l'un des suivants : {', '.join(valid_intervals)}.")
            return df

        # Resampler selon l'intervalle choisi et prendre la moyenne
        df_resampled = df.resample(resample_interval).mean().round(2)

        # Obtenir les dates min et max directement depuis l'index
        min_date, max_date = df_resampled.index.min(), df_resampled.index.max()

        # Afficher les dates min et max
        print(f"Plage de dates après resampling ({resample_interval}): \nMin: {min_date}\nMax: {max_date}  ")
        
        # Retourner la DataFrame résultante
        return df_resampled
    
    except ValueError as e:
        print(f"Erreur de conversion des dates : {e}")
        return df
    except KeyError as e:
        print(f"Erreur : La colonne spécifiée n'a pas été trouvée dans le DataFrame. ({e})")
        return df
    except Exception as e:
        print(f"Erreur inattendue : {e}")
        return df
#
def handle_null_values(df):
    # Calcul des pourcentages de valeurs manquantes
    missing_percent = round((df.isnull().sum() / len(df)) * 100, 2)
    missing_percent_str = missing_percent.astype(str) + '%'
    

    # Gestion des valeurs manquantes
    for column in df.columns:
        null_percentage = missing_percent[column]
        
        if null_percentage == 100:
            # Si 100% des valeurs sont manquantes, on supprime la colonne
            print(f"Supprime la colonne : {column} (100% de valeurs manquantes)")
            df = df.drop(column, axis=1)
        elif null_percentage > 50:
            # Si plus de 50% des valeurs sont manquantes, on supprime la colonne
            print(f"Supprime la colonne : {column} ({null_percentage}% de valeurs manquantes)")
            df = df.drop(column, axis=1)
        elif null_percentage > 0:
            # Si moins de 50% des valeurs sont manquantes, on impute avec la médiane
            print(f"Impute la colonne : {column} avec la médiane ({null_percentage}% de valeurs manquantes)")
            median_value = df[column].median()
            df[column] = df[column].fillna(median_value)
       
    # Retourner le DataFrame modifié
    return df
#
def meteo_api_request(coordinates, mode='historical', days=92, interval='hourly'):
    """
    Fonction pour récupérer les données météo depuis l'API Open-Meteo avec cache et réessayer en cas d'erreur.
    
    Paramètres:
        coordinates (list) : liste de coordonnées [latitude, longitude]
        mode (str) : intervalle de données ('forecast' ou 'historical', par défaut 'historical')
        days (int) : nombre de jours dans le passé ou dans le futur (par défaut : 92 pour historique)
        interval (str) : intervalle des données ('hourly' ou 'daily', par défaut 'hourly')

    Retourne :
        pd.DataFrame : un DataFrame avec les données météo
    """
    # Fonction utilitaire pour convertir les coordonnées avec ou sans suffixe (ex: '45.5W', '-45.5')
    def parse_coordinates(coord):
        # Vérifie si la coordonnée a un suffixe de direction (W, E, N, S)
        pattern = r"^([-+]?\d+(\.\d+)?)([NSEW]?)$"
        match = re.match(pattern, str(coord))
        if match:
            value = float(match.group(1))
            direction = match.group(3)
            
            if direction == 'W' or direction == 'S':
                value = -abs(value)  # Si direction est Ouest ou Sud, on inverse la valeur
            
            return value
        else:
            raise ValueError(f"Coordonnée invalide : {coord}")

    # Convertir les coordonnées
    latitude = parse_coordinates(coordinates[0])
    longitude = parse_coordinates(coordinates[1])

    # Setup de l'API client avec retry et cache
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    openmeteo = openmeteo_requests.Client(session=cache_session)

    url = "https://api.open-meteo.com/v1/forecast"
    
    # Paramètres de base
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "past_days": days if mode == 'historical' else None,  # Si historique, utiliser 'past_days'
        "forecast_days": days if mode == 'forecast' else None,  # Si forecast, utiliser 'forecast_days'
        "hourly" if interval.lower() == 'hourly' else "daily": [
            "temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", 
            "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
            "cloud_cover_high", "visibility", "wind_speed_10m", "soil_temperature_0cm", "soil_moisture_0_to_1cm", 
            "is_day"
        ]
    }

    # Faire l'appel API
    responses = openmeteo.weather_api(url, params=params)

    # Traiter la réponse pour le premier emplacement
    response = responses[0]  # On prend la première réponse si plusieurs lieux sont fournis

    # Initialisation du dictionnaire pour les données à retourner
    data = {}

    # Processus des données en fonction du mode sélectionné
    if mode == 'historical':
        # Traitement pour données historiques
        if interval == 'hourly':
            hourly = response.Hourly()
            hourly_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=hourly.Interval()),
                    inclusive="left"
                )
            }
            hourly_variables = [
                "temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", 
                "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
                "cloud_cover_high", "visibility", "wind_speed_10m", "soil_temperature_0cm", "soil_moisture_0_to_1cm", 
                "is_day"
            ]
            
            for i, var in enumerate(hourly_variables):
                hourly_data[var] = [round(value, 2) for value in hourly.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(hourly_data)

        elif interval == 'daily':
            daily = response.Daily()
            daily_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=daily.Interval()),
                    inclusive="left"
                )
            }
            daily_variables = [
                "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", 
                "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", 
                "precipitation_sum", "rain_sum", "showers_sum", "precipitation_hours", "precipitation_probability_max", 
                "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum"
            ]
            
            for i, var in enumerate(daily_variables):
                daily_data[var] = [round(value, 2) for value in daily.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(daily_data)

    # If Forecast is chosen
    elif mode == 'forecast':
        # Traitement pour prévisions
        if interval == 'hourly':
            hourly = response.Hourly()
            hourly_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=hourly.Interval()),
                    inclusive="left"
                )
            }
            hourly_variables = [
                "temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", 
                "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
                "cloud_cover_high", "visibility", "wind_speed_10m", "soil_temperature_0cm", "soil_moisture_0_to_1cm", 
                "is_day"
            ]
            
            for i, var in enumerate(hourly_variables):
                hourly_data[var] = [round(value, 2) for value in hourly.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(hourly_data)

        elif interval == 'daily':
            daily = response.Daily()
            daily_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=daily.Interval()),
                    inclusive="left"
                )
            }
            daily_variables = [
                "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", 
                "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", 
                "precipitation_sum", "rain_sum", "showers_sum", "precipitation_hours", "precipitation_probability_max", 
                "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum"
            ]
            
            for i, var in enumerate(daily_variables):
                daily_data[var] = [round(value, 2) for value in daily.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(daily_data)

# Fonction pour se connecter à PostgreSQL via psycopg2 pour la gestion de la DB
def connect_psycopg2(dbname, user, password, host="localhost", port="5432"):
    """Établit la connexion à PostgreSQL avec psycopg2 pour la base de données spécifiée."""
    try:
        conn = psycopg2.connect(
            dbname=dbname, user=user, password=password, host=host, port=port
        )
        print(f"Connexion réussie à la base {dbname}.")
        return conn
    except Exception as e:
        print(f"Erreur de connexion : {e}")
        return None


# Fonction pour créer une base de données si elle n'existe pas
def create_database(dbname, user, password, host, port):
    """Crée une base de données si elle n'existe pas déjà."""
    # Se connecter à la base 'postgres' pour créer d'autres DB
    conn = connect_psycopg2(user=user, password=password, host =host, port=port)
    if conn is None:
        return  # Si la connexion échoue, on arrête la fonction

    conn.autocommit = True  # Désactiver la transaction active
    cur = conn.cursor()
    
    # Vérifier si la base de données existe déjà
    cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{dbname}'")
    exists = cur.fetchone()

    if not exists:
        try:
            cur.execute(f"CREATE DATABASE {dbname}")
            print(f"✅ Base '{dbname}' créée.")
        except Exception as e:
            print(f"Erreur lors de la création de la base de données : {e}")
    else:
        print(f"ℹ️ La base '{dbname}' existe déjà.")

    cur.close()
    conn.close()


# Fonction pour se connecter à une base spécifique via SQLAlchemy
def connect_postgresql(dbname, user, password, host, port):
    """Se connecte à la base Postgresql via SQLAlchemy."""
    db_url = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"
    try:
        engine = create_engine(db_url)
        conn = engine.connect()
        print(f"Connexion réussie à la base {dbname} via SQLAlchemy.")
        return conn
    except Exception as e:
        print(f"Erreur de connexion avec SQLAlchemy : {e}")
        return None


# Fonction pour créer un schéma dans la base
def create_schema(conn, schema_name):
    """Crée un schéma dans la base si ce n'est pas déjà fait."""
    if conn is not None:
        try:
            conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema_name}"))
            print(f"✅ Schéma '{schema_name}' créé.")
        except Exception as e:
            print(f"Erreur lors de la création du schéma : {e}")

In [5]:
# Définir les scopes requis pour accéder à Google Sheets et Google Drive
scopes = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/drive.appdata','https://www.googleapis.com/auth/drive.appfolder'
]

# Chemin vers le fichier de crédentials JSON
path_to_google_cred = "c:/Users/f.gionnane/Downloads/Mes-Projets/google_sheets_credentials.json"

# Créer les crédentials à partir du fichier de service account
creds = Credentials.from_service_account_file(path_to_google_cred, scopes=scopes)

# Autoriser l'accès en utilisant les crédentials
client = gspread.authorize(creds)

# URL de la feuille Google Sheets
sheet_url = "https://docs.google.com/spreadsheets/d/1rXyD4WW5DcrO2iCywfwLbqps3Z-RumxQH9hJofhVM24/edit?gid=0"

# Ouvrir la feuille Google Sheets en utilisant son URL
spreadsheet = client.open_by_url(sheet_url)

worksheets = spreadsheet.worksheets()

drive_service = build('drive', 'v3', credentials=creds)

marine_data_sheet = spreadsheet.get_worksheet(0)  # Accéder à la première feuille (gid=0)
marine_data = marine_data_sheet.get_all_records()  # Récupérer toutes les données de la feuille

meteo_data_sheet = spreadsheet.get_worksheet(0)  # Accéder à la première feuille (gid=0)
meteo_data = marine_data_sheet.get_all_records()  # Récupérer toutes les données de la feuille

print(marine_data)  # Afficher les données

[]


Get Data from APIs

Marine API

In [6]:
from ndbc_api import NdbcApi
api = NdbcApi()

# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()

stations_df.head()

,Station,Hull No./Config and Location,Location Lat/Long,Wind Speed,Wind Direction,Sea Level Pressure,Wave Height,Dominant Period,Air Temp,Water Temp,Dew Point,Remark
0,41001,East Hatteras,34.70N 72.23W,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,"Buoy recovered 10/26/24, data release stopped."
1,41002,3DV33 (SC) South Hatteras,31.75N 74.93W,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Buoy adrift 12/30/24.
2,41004,3DV02 (SC) Edisto,32.50N 79.08W,100,100,100,100,100,100,100,79,Dewpoint is intermittent.
3,41008,3D36 (SC) Grays Reef,31.40N 80.85W,64,64,100,98,98,64,99,46,All data is intermittent after 8/18/23.
4,41009,3D65 (SC) Canaveral,28.50N 80.18W,100,100,100,100,100,100,99,100,Water temperature ceased 3/25/23. Air temperat...


Get Nearest

In [7]:
lat_SM = "11.24N"
lon_SM = "74.21W"

nearest = api.nearest_station(lat_SM,lon_SM)
nearest

'42058'

In [8]:
buoy_near_SM = api.station(station_id=nearest, as_df=False)
buoy_near_SM

{'Watch circle radius': '4123 yards',
 'Water depth': '4136 m',
 'Sea temp depth': '1.5 m below water line',
 'Barometer elevation': '2.7 m above mean sea level',
 'Anemometer height': '4.1 m above site elevation',
 'Air temp height': '3.7 m above site elevation',
 'Site elevation': 'sea level',
 'Location': '14.512 N 75.153 W (14°30\'43" N 75°9\'11" W)',
 'Statation Type': 'Owned and maintained by National Data Buoy Center, 3-meter foam buoy, SCOOP payload',
 'Name': 'Station 42058  - Central Caribbean - 210 NM SSE of Kingston, Jamaica'}

In [9]:
from geopy.distance import geodesic
lat_SM = 11.24
lon_SM = -74.21
lat_buoy_near_SM = 14.51 
lon_buoy_near_SM = -75.15

coord_buoy_1 = (lat_SM, lon_SM)
coord_buoy_2 =(lat_buoy_near_SM, lon_buoy_near_SM)


distance_km = geodesic(coord_buoy_1,coord_buoy_2).kilometers

m = folium.Map(location=[lat_SM,lon_SM], zoom_start=6)

folium.Marker(location=[lat_buoy_near_SM,lon_buoy_near_SM], popup=f"Buoy nearest to Santa marta ({round(distance_km,2)}km)").add_to(m)
folium.Marker(location=[lat_SM,lon_SM], popup="Santa marta").add_to(m)
m

In [10]:
df_marine = NDBC.realtime_observations(nearest)
# Afficher le résultat
print(df_marine.shape)
df_marine.head()

(6513, 15)


,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean,time
0,70.0,9.0,10.0,NaN,NaN,NaN,NaN,1014.2,27.2,27.4,23.2,NaN,NaN,NaN,2025-02-21 17:10:00+00:00
1,70.0,9.0,11.0,NaN,NaN,NaN,NaN,1014.3,27.2,27.4,22.9,NaN,-1.1,NaN,2025-02-21 17:00:00+00:00
2,70.0,9.0,11.0,1.4,8.0,4.4,73.0,1014.5,27.3,27.4,23.0,NaN,NaN,NaN,2025-02-21 16:50:00+00:00
3,70.0,9.0,11.0,1.4,NaN,4.4,73.0,1014.6,27.2,27.4,23.0,NaN,NaN,NaN,2025-02-21 16:40:00+00:00
4,70.0,9.0,11.0,NaN,NaN,NaN,NaN,1014.8,27.2,27.4,23.0,NaN,NaN,NaN,2025-02-21 16:30:00+00:00


In [11]:
# Exemple d'utilisation avec l'intervalle par défaut 'h' (heure)
df_resampled = process_and_resample(df_marine, 'time')  # Intervalle par défaut 'h'
df_resampled.head()

Erreur inattendue : Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'


,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean,Datetime
0,70.0,9.0,10.0,NaN,NaN,NaN,NaN,1014.2,27.2,27.4,23.2,NaN,NaN,NaN,2025-02-21 17:10:00+00:00
1,70.0,9.0,11.0,NaN,NaN,NaN,NaN,1014.3,27.2,27.4,22.9,NaN,-1.1,NaN,2025-02-21 17:00:00+00:00
2,70.0,9.0,11.0,1.4,8.0,4.4,73.0,1014.5,27.3,27.4,23.0,NaN,NaN,NaN,2025-02-21 16:50:00+00:00
3,70.0,9.0,11.0,1.4,NaN,4.4,73.0,1014.6,27.2,27.4,23.0,NaN,NaN,NaN,2025-02-21 16:40:00+00:00
4,70.0,9.0,11.0,NaN,NaN,NaN,NaN,1014.8,27.2,27.4,23.0,NaN,NaN,NaN,2025-02-21 16:30:00+00:00


In [12]:
"""

#### Prise en charge de plusieurs buoys à la fois

caribbean_df = stations_df[(stations_df['Lat'] >= 9) & 
                           (stations_df['Lat'] <= 25) & 
                           (stations_df['Lon'] >= -85) & 
                           (stations_df['Lon'] <= -60)]
caribbean_df.shape[0]

buoys_ids = []

for index, row in caribbean_df.iterrows():
    buoys_ids.append(row['Station'])  
print(len(buoys_ids))    

# Liste pour stocker les dataframes
all_dataframes = []

# Compteurs pour les bouées réussies et échouées
successful_buoy = 0
failed_buoy = 0

# Parcours de chaque bouée
for buoy_id in buoys_ids:
    try:
        # Récupère les observations en temps réel pour chaque bouée
        df_buoy = NDBC.realtime_observations(buoy_id)

        # Ajoute le dataframe à la liste
        all_dataframes.append(df_buoy)
        successful_buoy += 1

        # Efface et met à jour la ligne des bouées réussies
        print(f"\r✅ Bouées réussies : {successful_buoy}", end='', flush=True)

    except Exception:
        # Incrémente le compteur des échouées pour toutes les erreurs
        failed_buoy += 1

        # Efface et met à jour la ligne des bouées échouées
        print(f"\r❌ Bouées échouées : {failed_buoy}", end='', flush=True)

# Affichage final propre avec un saut de ligne
print("\n")

# Concaténation des dataframes si disponibles
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"✔️ Concaténation terminée. Nombre de lignes : {final_df.shape[0]} \n Nombre de colonnes : {final_df.shape[1]}")
    print(f"✅ Bouées réussies : {successful_buoy}", end='', flush=True)
    print(f"❌ Bouées échouées : {failed_buoy}", end='', flush=True)

else:
    print("⚠️ Aucune donnée récupérée.")
    
"""

'\n\n#### Prise en charge de plusieurs buoys à la fois\n\ncaribbean_df = stations_df[(stations_df[\'Lat\'] >= 9) & \n                           (stations_df[\'Lat\'] <= 25) & \n                           (stations_df[\'Lon\'] >= -85) & \n                           (stations_df[\'Lon\'] <= -60)]\ncaribbean_df.shape[0]\n\nbuoys_ids = []\n\nfor index, row in caribbean_df.iterrows():\n    buoys_ids.append(row[\'Station\'])  \nprint(len(buoys_ids))    \n\n# Liste pour stocker les dataframes\nall_dataframes = []\n\n# Compteurs pour les bouées réussies et échouées\nsuccessful_buoy = 0\nfailed_buoy = 0\n\n# Parcours de chaque bouée\nfor buoy_id in buoys_ids:\n    try:\n        # Récupère les observations en temps réel pour chaque bouée\n        df_buoy = NDBC.realtime_observations(buoy_id)\n\n        # Ajoute le dataframe à la liste\n        all_dataframes.append(df_buoy)\n        successful_buoy += 1\n\n        # Efface et met à jour la ligne des bouées réussies\n        print(f"\r✅ Bouées ré

In [13]:
# Exemple d'appel de la fonction
df_marine = handle_null_values(df_marine)

print(df_marine.shape)
df_marine.head()

Impute la colonne : wind_direction avec la médiane (0.09% de valeurs manquantes)
Impute la colonne : wind_speed avec la médiane (0.09% de valeurs manquantes)
Impute la colonne : wind_gust avec la médiane (0.09% de valeurs manquantes)
Impute la colonne : wave_height avec la médiane (43.16% de valeurs manquantes)
Supprime la colonne : dominant_wave_period (66.99% de valeurs manquantes)
Impute la colonne : average_wave_period avec la médiane (43.18% de valeurs manquantes)
Impute la colonne : dominant_wave_direction avec la médiane (43.18% de valeurs manquantes)
Impute la colonne : pressure avec la médiane (0.05% de valeurs manquantes)
Impute la colonne : air_temperature avec la médiane (0.84% de valeurs manquantes)
Impute la colonne : water_temperature avec la médiane (1.15% de valeurs manquantes)
Impute la colonne : dewpoint avec la médiane (0.84% de valeurs manquantes)
Supprime la colonne : visibility (100% de valeurs manquantes)
Supprime la colonne : 3hr_pressure_tendency (83.26% de va

,wind_direction,wind_speed,wind_gust,wave_height,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,Datetime
0,70.0,9.0,10.0,2.4,5.7,81.0,1014.2,27.2,27.4,23.2,2025-02-21 17:10:00+00:00
1,70.0,9.0,11.0,2.4,5.7,81.0,1014.3,27.2,27.4,22.9,2025-02-21 17:00:00+00:00
2,70.0,9.0,11.0,1.4,4.4,73.0,1014.5,27.3,27.4,23.0,2025-02-21 16:50:00+00:00
3,70.0,9.0,11.0,1.4,4.4,73.0,1014.6,27.2,27.4,23.0,2025-02-21 16:40:00+00:00
4,70.0,9.0,11.0,2.4,5.7,81.0,1014.8,27.2,27.4,23.0,2025-02-21 16:30:00+00:00


open-meteo API

In [14]:
coordinates = [lat_buoy_near_SM, lon_buoy_near_SM]
df_meteo = meteo_api_request(coordinates=coordinates)
print(df_meteo.shape)
df_meteo.head()

(2376, 18)


,date,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-11-21 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2024-11-21 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2024-11-21 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2024-11-21 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,2024-11-21 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [15]:
df_meteo =process_and_resample(df_meteo, 'date')
df_meteo.head()

Erreur inattendue : Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'


,Datetime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-11-21 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2024-11-21 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2024-11-21 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2024-11-21 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,2024-11-21 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [16]:
# Arrondir toutes les colonnes numériques à 2 décimales
df_meteo.loc[:, df_meteo.select_dtypes(include=['float32', 'float64']).columns] = df_meteo.select_dtypes(include=['float32', 'float64']).applymap(lambda x: round(x, 2))
df_meteo.head()

,Datetime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-11-21 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2024-11-21 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2024-11-21 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2024-11-21 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,2024-11-21 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [17]:
df_meteo = handle_null_values(df_meteo)
df_marine = handle_null_values(df_marine)
print(df_marine.columns)
print(df_meteo.columns)
df_marine.head()

Impute la colonne : temperature_2m avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : relative_humidity_2m avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : dew_point_2m avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : precipitation avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : rain avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : showers avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : pressure_msl avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : surface_pressure avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : cloud_cover avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : cloud_cover_low avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : cloud_cover_mid avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : cloud_cover_high avec la médiane (12.92% de valeurs manquantes)
Impute la colonne : visibilit

,wind_direction,wind_speed,wind_gust,wave_height,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,Datetime
0,70.0,9.0,10.0,2.4,5.7,81.0,1014.2,27.2,27.4,23.2,2025-02-21 17:10:00+00:00
1,70.0,9.0,11.0,2.4,5.7,81.0,1014.3,27.2,27.4,22.9,2025-02-21 17:00:00+00:00
2,70.0,9.0,11.0,1.4,4.4,73.0,1014.5,27.3,27.4,23.0,2025-02-21 16:50:00+00:00
3,70.0,9.0,11.0,1.4,4.4,73.0,1014.6,27.2,27.4,23.0,2025-02-21 16:40:00+00:00
4,70.0,9.0,11.0,2.4,5.7,81.0,1014.8,27.2,27.4,23.0,2025-02-21 16:30:00+00:00


In [18]:
df_meteo = drop_columns_if_exist(df_meteo,['rain', 'showers','soil_moisture_0_to_1cm', 'cloud_cover', 'soil_temperature_0cm',	'soil_moisture_0_to_1cm', 'is_day'])
df_meteo.head()

Colonne 'rain' Supprimée
Colonne 'showers' Supprimée
Colonne 'soil_moisture_0_to_1cm' Supprimée
Colonne 'cloud_cover' Supprimée
Colonne 'soil_temperature_0cm' Supprimée
Colonne 'soil_moisture_0_to_1cm' Supprimée
Colonne 'is_day' Supprimée


,Datetime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,pressure_msl,surface_pressure,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m
0,2024-11-21 00:00:00+00:00,27.5,80.0,23.5,0.0,1012.7,1012.7,49.0,0.0,3.0,24140.0,36.32
1,2024-11-21 01:00:00+00:00,27.5,80.0,23.5,0.0,1012.7,1012.7,49.0,0.0,3.0,24140.0,36.32
2,2024-11-21 02:00:00+00:00,27.5,80.0,23.5,0.0,1012.7,1012.7,49.0,0.0,3.0,24140.0,36.32
3,2024-11-21 03:00:00+00:00,27.5,80.0,23.5,0.0,1012.7,1012.7,49.0,0.0,3.0,24140.0,36.32
4,2024-11-21 04:00:00+00:00,27.5,80.0,23.5,0.0,1012.7,1012.7,49.0,0.0,3.0,24140.0,36.32


In [19]:
df_meteo.rename(columns={'temperature_2m': 'T°(C°)', 
                         'relative_humidity_2m': 'Relative Humidity (%)',
                         'dew_point_2m': 'Dew Point (°C)', 
                         'precipitation': 'Precipitation (mm)', 
                         'pressure_msl':' Sea Level Pressure (hPa)', 
                         'cloud_cover_low':'Low Clouds (%)',
                         'cloud_cover_mid' : 'Middle Clouds (%)',	
                         'cloud_cover_high' : 'High Clouds (%)', 
                         'visibility' : ' Visibility (%)', 
                         'wind_speed_10m' : 'Wind Speed (km/h)'}, 
                         inplace=True)
df_marine.rename(columns={
    'wind_direction': 'Wind Direction (°)',
    'wind_speed': 'Wind Speed (km/h)',
    'wind_gust': 'Wind Gusts (km/h)',
    'wave_height': 'Wave Height (m)',
    'average_wave_period': 'Average Wave Period (s)',
    'dominant_wave_direction': 'Dominant Wave Direction (°)',
    'pressure': 'Pressure (hPA)',
    'air_temperature': 'Air T°',
    'water_temperature': 'Water T°'}, 
    inplace=True)

print(df_meteo.columns)
print(df_marine.columns)
print(df_marine.shape)
print(df_meteo.shape)

Index(['Datetime', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
       'Precipitation (mm)', ' Sea Level Pressure (hPa)', 'surface_pressure',
       'Low Clouds (%)', 'Middle Clouds (%)', 'High Clouds (%)',
       ' Visibility (%)', 'Wind Speed (km/h)'],
      dtype='object')
Index(['Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPA)', 'Air T°', 'Water T°',
       'dewpoint', 'Datetime'],
      dtype='object')
(6513, 11)
(2376, 12)


Merging Dataframes

In [20]:
# Effectuer la jointure interne sur la colonne 'time'
df_merged = pd.merge(df_marine, df_meteo, on = 'Datetime', how='inner')

# Afficher le résultat
print(df_merged.shape)
df_merged.head()

(1097, 22)


,Wind Direction (°),Wind Speed (km/h)_x,Wind Gusts (km/h),Wave Height (m),Average Wave Period (s),Dominant Wave Direction (°),Pressure (hPA),Air T°,Water T°,dewpoint,...,Relative Humidity (%),Dew Point (°C),Precipitation (mm),Sea Level Pressure (hPa),surface_pressure,Low Clouds (%),Middle Clouds (%),High Clouds (%),Visibility (%),Wind Speed (km/h)_y
0,70.0,9.0,11.0,2.4,5.7,81.0,1014.3,27.2,27.4,22.9,...,79.0,23.00,0.1,1015.1,1015.1,3.0,0.0,0.0,24140.0,31.21
1,70.0,9.0,11.0,2.4,5.7,81.0,1015.3,27.2,27.4,23.2,...,79.0,22.90,0.1,1015.7,1015.7,42.0,8.0,0.0,24140.0,31.21
2,70.0,9.0,11.0,2.4,5.7,81.0,1015.7,27.3,27.4,23.4,...,75.0,22.33,0.0,1016.3,1016.3,34.0,0.0,0.0,24140.0,33.25
3,80.0,9.0,11.0,2.4,5.7,81.0,1015.4,27.3,27.4,23.1,...,75.0,22.24,0.0,1015.9,1015.9,34.0,0.0,0.0,24140.0,33.71
4,80.0,9.0,10.0,2.4,5.7,81.0,1014.8,27.2,27.3,23.2,...,75.0,22.24,0.0,1015.1,1015.1,42.0,0.0,0.0,24140.0,35.18


In [21]:
print(df_merged.dtypes)
df_merged.head(20)

Wind Direction (°)                         float64
Wind Speed (km/h)_x                        float64
Wind Gusts (km/h)                          float64
Wave Height (m)                            float64
Average Wave Period (s)                    float64
Dominant Wave Direction (°)                float64
Pressure (hPA)                             float64
Air T°                                     float64
Water T°                                   float64
dewpoint                                   float64
Datetime                       datetime64[ns, UTC]
T°(C°)                                     float64
Relative Humidity (%)                      float32
Dew Point (°C)                             float64
Precipitation (mm)                         float64
 Sea Level Pressure (hPa)                  float64
surface_pressure                           float64
Low Clouds (%)                             float32
Middle Clouds (%)                          float32
High Clouds (%)                

,Wind Direction (°),Wind Speed (km/h)_x,Wind Gusts (km/h),Wave Height (m),Average Wave Period (s),Dominant Wave Direction (°),Pressure (hPA),Air T°,Water T°,dewpoint,...,Relative Humidity (%),Dew Point (°C),Precipitation (mm),Sea Level Pressure (hPa),surface_pressure,Low Clouds (%),Middle Clouds (%),High Clouds (%),Visibility (%),Wind Speed (km/h)_y
0,70.0,9.0,11.0,2.4,5.7,81.0,1014.3,27.2,27.4,22.9,...,79.0,23.00,0.1,1015.1,1015.1,3.0,0.0,0.0,24140.0,31.21
1,70.0,9.0,11.0,2.4,5.7,81.0,1015.3,27.2,27.4,23.2,...,79.0,22.90,0.1,1015.7,1015.7,42.0,8.0,0.0,24140.0,31.21
2,70.0,9.0,11.0,2.4,5.7,81.0,1015.7,27.3,27.4,23.4,...,75.0,22.33,0.0,1016.3,1016.3,34.0,0.0,0.0,24140.0,33.25
3,80.0,9.0,11.0,2.4,5.7,81.0,1015.4,27.3,27.4,23.1,...,75.0,22.24,0.0,1015.9,1015.9,34.0,0.0,0.0,24140.0,33.71
4,80.0,9.0,10.0,2.4,5.7,81.0,1014.8,27.2,27.3,23.2,...,75.0,22.24,0.0,1015.1,1015.1,42.0,0.0,0.0,24140.0,35.18
5,80.0,10.0,12.0,2.4,5.7,81.0,1014.1,27.1,27.3,22.5,...,77.0,22.53,0.0,1014.5,1014.5,13.0,0.0,0.0,24140.0,35.71
6,70.0,10.0,12.0,2.4,5.7,81.0,1013.1,27.0,27.2,22.9,...,77.0,22.53,0.0,1013.5,1013.5,36.0,1.0,0.0,24140.0,35.31
7,70.0,10.0,12.0,2.4,5.7,81.0,1012.4,27.1,27.2,22.9,...,77.0,22.48,0.0,1013.1,1013.1,39.0,0.0,0.0,24140.0,35.10
8,70.0,9.0,11.0,2.4,5.7,81.0,1012.1,27.0,27.2,22.7,...,76.0,22.26,0.0,1012.8,1012.8,41.0,0.0,0.0,24140.0,35.50
9,70.0,9.0,11.0,2.4,5.7,81.0,1011.7,27.0,27.8,22.9,...,76.0,22.36,0.0,1012.9,1012.9,35.0,0.0,0.0,24140.0,35.99


In [22]:
# Exemple d'utilisation
df_merged = add_daytime_and_month_column(df_merged,'Datetime')

print(df_merged.columns)
df_merged.head(10)

Index(['Wind Direction (°)', 'Wind Speed (km/h)_x', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPA)', 'Air T°', 'Water T°',
       'dewpoint', 'Datetime', 'T°(C°)', 'Relative Humidity (%)',
       'Dew Point (°C)', 'Precipitation (mm)', ' Sea Level Pressure (hPa)',
       'surface_pressure', 'Low Clouds (%)', 'Middle Clouds (%)',
       'High Clouds (%)', ' Visibility (%)', 'Wind Speed (km/h)_y', 'DayTime',
       'Month'],
      dtype='object')


,Wind Direction (°),Wind Speed (km/h)_x,Wind Gusts (km/h),Wave Height (m),Average Wave Period (s),Dominant Wave Direction (°),Pressure (hPA),Air T°,Water T°,dewpoint,...,Precipitation (mm),Sea Level Pressure (hPa),surface_pressure,Low Clouds (%),Middle Clouds (%),High Clouds (%),Visibility (%),Wind Speed (km/h)_y,DayTime,Month
0,70.0,9.0,11.0,2.4,5.7,81.0,1014.3,27.2,27.4,22.9,...,0.1,1015.1,1015.1,3.0,0.0,0.0,24140.0,31.21,Afternoon,2
1,70.0,9.0,11.0,2.4,5.7,81.0,1015.3,27.2,27.4,23.2,...,0.1,1015.7,1015.7,42.0,8.0,0.0,24140.0,31.21,Afternoon,2
2,70.0,9.0,11.0,2.4,5.7,81.0,1015.7,27.3,27.4,23.4,...,0.0,1016.3,1016.3,34.0,0.0,0.0,24140.0,33.25,Afternoon,2
3,80.0,9.0,11.0,2.4,5.7,81.0,1015.4,27.3,27.4,23.1,...,0.0,1015.9,1015.9,34.0,0.0,0.0,24140.0,33.71,Afternoon,2
4,80.0,9.0,10.0,2.4,5.7,81.0,1014.8,27.2,27.3,23.2,...,0.0,1015.1,1015.1,42.0,0.0,0.0,24140.0,35.18,Afternoon,2
5,80.0,10.0,12.0,2.4,5.7,81.0,1014.1,27.1,27.3,22.5,...,0.0,1014.5,1014.5,13.0,0.0,0.0,24140.0,35.71,Afternoon,2
6,70.0,10.0,12.0,2.4,5.7,81.0,1013.1,27.0,27.2,22.9,...,0.0,1013.5,1013.5,36.0,1.0,0.0,24140.0,35.31,Morning,2
7,70.0,10.0,12.0,2.4,5.7,81.0,1012.4,27.1,27.2,22.9,...,0.0,1013.1,1013.1,39.0,0.0,0.0,24140.0,35.10,Morning,2
8,70.0,9.0,11.0,2.4,5.7,81.0,1012.1,27.0,27.2,22.7,...,0.0,1012.8,1012.8,41.0,0.0,0.0,24140.0,35.50,Morning,2
9,70.0,9.0,11.0,2.4,5.7,81.0,1011.7,27.0,27.8,22.9,...,0.0,1012.9,1012.9,35.0,0.0,0.0,24140.0,35.99,Morning,2


In [23]:
df_merged['Wind Speed (km/h)'] = (df_merged['Wind Speed (km/h)_x']+ df_merged['Wind Speed (km/h)_y'])/2
df_merged = drop_columns_if_exist(df_merged, ['Wind Speed (km/h)_x', 'Wind Speed (km/h)_y', 'Wind Gusts (km/h)'])

df_merged.columns
df_merged.head()

Colonne 'Wind Speed (km/h)_x' Supprimée
Colonne 'Wind Speed (km/h)_y' Supprimée
Colonne 'Wind Gusts (km/h)' Supprimée


,Wind Direction (°),Wave Height (m),Average Wave Period (s),Dominant Wave Direction (°),Pressure (hPA),Air T°,Water T°,dewpoint,Datetime,T°(C°),...,Precipitation (mm),Sea Level Pressure (hPa),surface_pressure,Low Clouds (%),Middle Clouds (%),High Clouds (%),Visibility (%),DayTime,Month,Wind Speed (km/h)
0,70.0,2.4,5.7,81.0,1014.3,27.2,27.4,22.9,2025-02-21 17:00:00+00:00,26.95,...,0.1,1015.1,1015.1,3.0,0.0,0.0,24140.0,Afternoon,2,20.105
1,70.0,2.4,5.7,81.0,1015.3,27.2,27.4,23.2,2025-02-21 16:00:00+00:00,26.85,...,0.1,1015.7,1015.7,42.0,8.0,0.0,24140.0,Afternoon,2,20.105
2,70.0,2.4,5.7,81.0,1015.7,27.3,27.4,23.4,2025-02-21 15:00:00+00:00,27.15,...,0.0,1016.3,1016.3,34.0,0.0,0.0,24140.0,Afternoon,2,21.125
3,80.0,2.4,5.7,81.0,1015.4,27.3,27.4,23.1,2025-02-21 14:00:00+00:00,27.05,...,0.0,1015.9,1015.9,34.0,0.0,0.0,24140.0,Afternoon,2,21.355
4,80.0,2.4,5.7,81.0,1014.8,27.2,27.3,23.2,2025-02-21 13:00:00+00:00,27.05,...,0.0,1015.1,1015.1,42.0,0.0,0.0,24140.0,Afternoon,2,22.090


In [70]:
postgres_creds_path = "C:/Users/f.gionnane/Downloads/Mes-Projets/Oceanography-Project-WIP/postgresql_creds.json"

with open(postgres_creds_path, 'r') as file:
            content = json.load(file)

host='localhost'
port=5432
user='flosrv'
password='Nesrine123@'
dbname='myprojects'
print(port)
conn = connect_postgresql('myprojects', user=user, password=password, host=host, port=port)


5432
Erreur de connexion avec SQLAlchemy : (psycopg2.OperationalError) connection to server on socket "@localhost/.s.PGSQL.5432" failed: Invalid argument (0x00002726/10022)
	Is the server running locally and accepting connections on that socket?

(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
import psycopg2
import json

postgres_creds_path = "C:/Users/f.gionnane/Downloads/Mes-Projets/Oceanography-Project-WIP/postgresql_creds.json"

def create_or_connect_postgres_db(db_name):
    try:
        # Charger les identifiants depuis le fichier JSON
        with open(postgres_creds_path, 'r') as file:
            content = json.load(file)
        # Connexion initiale sans spécifier la base de données
        conn = psycopg2.connect(
            host=content['host'],
            port=content['Port'],
            user=content['User'],
            password=content['Password'],
            dbname='myprojects'
        )
        conn.autocommit = True
        cursor = conn.cursor()

        # Vérifier si la base de données existe
        cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s", (db_name,))
        exists = cursor.fetchone()

        if not exists:
            cursor.execute(f'CREATE DATABASE "{db_name}"')
            print(f"✅ Base de données '{db_name}' créée avec succès.")
        else:
            print(f"ℹ️  La base de données '{db_name}' existe déjà. Connexion en cours...")

        cursor.close()
        conn.close()

        # Connexion à la base de données cible
        conn = psycopg2.connect(
            host=content['host'],
            port=content['port'],
            user=content['user'],
            password=content['password'],
            dbname=db_name
        )
        print(f"🚀 Connecté à la base de données '{db_name}' avec succès.")
        return conn

    except psycopg2.Error as e:
        print(f"❌ Erreur PostgreSQL : {e.pgerror}")
    except FileNotFoundError:
        print("❌ Fichier des identifiants non trouvé.")
    except json.JSONDecodeError:
        print("❌ Erreur lors de la lecture du fichier JSON.")
    except Exception as e:
        print(f"❌ Erreur inattendue : {e}")

    return None

In [ ]:
def insert_df_to_table(df, table_name, schema_name, conn, if_exists="append", unique_columns=None):

    # Vérifier si la table existe déjà
    metadata = MetaData()
    table = Table(table_name, metadata, autoload_with=conn, schema=schema_name)

    if table.exists(conn):
        print(f"ℹ️ La table '{table_name}' existe déjà. Ajout des données en mode '{if_exists}'.")
    else:
        print(f"✅ La table '{table_name}' n'existe pas. Création de la table.")
        # Créer la table à partir de la structure du DataFrame
        df.head(0).to_sql(table_name, conn, schema=schema_name, if_exists='replace', index=False)
    
    # Gestion de l'insertion des données
    try:
        if unique_columns:
            # Utilisation de 'on conflict' pour ignorer les doublons, basé sur des colonnes uniques
            for _, row in df.iterrows():
                insert_query = f"""
                INSERT INTO {schema_name}.{table_name} ({', '.join(df.columns)})
                VALUES ({', '.join([repr(val) for val in row])})
                ON CONFLICT ({', '.join(unique_columns)}) DO NOTHING;
                """
                conn.execute(insert_query)
        else:
            # Insertion standard sans gestion des doublons
            df.to_sql(table_name, conn, schema=schema_name, if_exists=if_exists, index=False)
        print("✅ Données insérées avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion des données : {e}")